In [1]:
import os, sys
sys.path.append('..') # Parent directory in path
from time import time, sleep
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
import scipy.stats as stats
from scipy.interpolate import interp1d
from scipy.optimize import curve_fit
import statsmodels.api as sm

pd.set_option("display.precision", 4)
#import pandas_market_calendars as mcal # NYSE Calendar

import matplotlib.pyplot as plt
plt.style.use('seaborn')
plt.rc("font", **{"size": 14})
plt.rc("figure", **{"figsize": (16,10)})
# import matplotlib.pylab as pl
from matplotlib import cm

In [2]:
def hyperbola(x, a, b, c, d, e) -> float:
    """Tiltable hyperbola for scipy.optimize.curve_fit
    Asymptotes at 
    y=(-b/a+d)(x-c)+e
    y=( b/a+d)(x-c)+e"""
    return (b**2*(1+(x-c)**2/a**2))**0.5+d*(x-c)+e

def hyperbola_jac(x, a, b, c, d, e) -> np.ndarray:
    """Jacobian of the hyperbola with respect to each parameter"""
    ru = np.sqrt(b**2*(1+(x-c)**2/a**2))
    da = -(b**2 * (x-c)**2) / (a**3 * ru)
    db = np.sqrt(1+(x-c)**2/a**2)
    dc = -(b**2 * (x-c)) / (a**2 * ru) - d
    dd = x-c
    de = np.ones(len(x))
    return np.array([da, db, dc, dd, de]).T

def hyperbola_minimizer(a, b, c, d, e) -> float:
    """Returns the minimizer (x such that y is the minimum) of a hyperbola"""
    return ((a**2 * c * d**2 + d*(a**4 * b**2 - a**6 * d**2)**0.5 - b**2 * c)
            /(a**2 * d**2 - b**2))

def fit_hyp(df) -> np.ndarray:
    """Fits a hyperbola for a day's IV single curve"""
    curve = df.loc[:,["LOG_MONEYNESS_F","IV_MID"]].dropna()
    bounds = ((1e-4, 1e-4, 0, -2, -2), (1, 1, 1, 2, 2))
    try:
        popt, pcov = curve_fit(hyperbola, curve["LOG_MONEYNESS_F"], curve["IV_MID"]**2, 
                               jac=hyperbola_jac, bounds=bounds, maxfev=5000)
        return popt
    except (RuntimeError, ValueError):
        return np.array([np.nan] * 5)

In [3]:
all = pd.concat([pd.read_hdf(os.path.join("..", "data", "spx_iv_db_1.h5")), 
                 pd.read_hdf(os.path.join("..", "data", "spx_iv_db_2.h5"))])

exp_dt = all['EXP'].unique()
writer = pd.ExcelWriter('./iv_parameter.xlsx', engine = 'xlsxwriter')

In [4]:
for dt in exp_dt:
    print(dt)
    df = all[all['EXP'] == dt].dropna().reset_index(drop=True)
    ATM_vols = df[df["RANK"]==0]
    df = pd.merge(df, ATM_vols[["TS", "TYPE", "IV_BID", "IV_ASK"]], left_on=["TS", "TYPE"], right_on=["TS", "TYPE"], 
                  suffixes=(None, "_ATM"), how="left")
    
    df["MONEYNESS"] = df["UNDERLYING_PRICE"] - df["STRIKE"]
    df["MONEYNESS_F"] = df["STRIKE"]/df["F_T"]
    df["LOG_MONEYNESS_F"] = np.log(df["MONEYNESS_F"])
    df["LOG_MONEYNESS_F_STANDARD_TIME"] = df["LOG_MONEYNESS_F"] / np.sqrt(df["CAL_DAYS"])

    # df["LOG_MONEYNESS_F_STANDARD_STD"] = df["LOG_MONEYNESS_F"] / (np.sqrt(df["CAL_DAYS"])*(df["IV_BID_ATM"]+df["IV_ASK_ATM"])/2)

    df["IV_BID_DIFF"] = df["IV_BID"] - df["IV_BID_ATM"]
    df["IV_ASK_DIFF"] = df["IV_ASK"] - df["IV_ASK_ATM"]

    df["IV_BID_RATIO"] = df["IV_BID"]/df["IV_BID_ATM"]
    df["IV_ASK_RATIO"] = df["IV_ASK"]/df["IV_ASK_ATM"]
    
    otm_mask = (((df["TYPE"]=='C') & (df["STRIKE"]>df["UNDERLYING_PRICE"])) 
            | ((df["TYPE"]=='P') & (df["STRIKE"]<=df["UNDERLYING_PRICE"])))

    otm  = df[otm_mask].copy().reset_index(drop=True)
    to_fit = otm

    fits = to_fit.groupby("TS").apply(fit_hyp)
    fits = np.array([v for v in fits.values])

    fit_df = pd.DataFrame(fits, index=to_fit["TS"].unique(), columns=['a', 'b', 'c', 'd', 'e'])
    fit_df.to_excel(writer, sheet_name=pd.to_datetime(str(dt)).strftime('%Y-%m-%d'))

2017-07-21T00:00:00.000000000
2017-08-18T00:00:00.000000000
2017-09-15T00:00:00.000000000
2017-10-20T00:00:00.000000000
2017-11-17T00:00:00.000000000
2017-12-15T00:00:00.000000000
2018-01-19T00:00:00.000000000
2018-02-16T00:00:00.000000000
2018-03-16T00:00:00.000000000
2018-04-20T00:00:00.000000000
2018-05-18T00:00:00.000000000
2018-06-15T00:00:00.000000000
2018-07-20T00:00:00.000000000
2018-08-17T00:00:00.000000000
2018-09-21T00:00:00.000000000
2018-10-19T00:00:00.000000000
2018-11-16T00:00:00.000000000
2018-12-21T00:00:00.000000000
2019-01-18T00:00:00.000000000
2019-02-15T00:00:00.000000000
2019-03-15T00:00:00.000000000
2019-04-18T00:00:00.000000000
2019-05-17T00:00:00.000000000
2019-06-21T00:00:00.000000000
2019-07-19T00:00:00.000000000
2019-08-16T00:00:00.000000000
2019-09-20T00:00:00.000000000
2019-10-18T00:00:00.000000000
2019-11-15T00:00:00.000000000
2019-12-20T00:00:00.000000000
2020-01-17T00:00:00.000000000
2020-02-21T00:00:00.000000000
2020-03-20T00:00:00.000000000
2020-04-17

In [5]:
writer.close()